In [ ]:
#r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Debug\net5.0\bossspad.dll"

//#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Open Database

In [ ]:
BoSSSshell.WorkflowMgm.Init("CoFloFlame_allOnRefined_F33_ahorasdddd2");

In [ ]:
static var myBatch = BoSSSshell.ExecutionQueues[2];
// static var myBatch = GetDefaultQueue();
// static var myDb = OpenOrCreateDatabase(@"C:\Databases\BoSSSAAA");
// static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\CoFloFlame_allOnRefined_timevariantBC_MF");
static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\CoFloFlame_allOnRefinedd1dddddd_complexchemmodel");

// static var myDb = OpenOrCreateDatabase(@"C:\Databases\123121442");

// static var myDb = myBatch.CreateOrOpenCompatibleDatabase("CoFlowFlame5");
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation()

## Case configuration

In [ ]:
int NumberOfProcMF = 8;
int nCells = 11; // cell mult factor

// int nCells = 10; // cell mult factor
int dgMF =2; // Mass fraction DG degree
int dgFull = 4;
double[] smoothingFactors = new double[] {0,50};
double InitialMassFuelIn = 0.002400*10; //kg/m2s 
double InitialMassAirIn = 0.002400 *10*2; //kg/m2s 

bool calculateOnlyMixtureFraction = false;

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid_FullSystem(int nCells) {
         // Geometry
        // All lenghts are non.dimensionalized with Lref = fuel inlet radius (0.2 cm)
        double factor = 1.0;

        double r = 0.635 / 100 * factor; // Radius inner cylinder, m
        double delta = r / 5;
        double R = 2.54 / 100 * factor * 3 + delta; // Radius outter cylinder, m
        double LRef = r;

        double zlength = 20.0 / 100 * 2;// meters
        double xmin = -R / LRef;
        double xmax = +R / LRef;
        double ymin = 0;
        double ymax = zlength / LRef;
        double rAd = r / LRef;
        double deltaNonDim = delta / LRef;

        double leftmidpoint = (xmin - rAd) / 2;
        double rightmidpoint = (xmax + rAd) / 2;

    
        double stretchfactorY = 0.98 * 1;

        double sf1 = 0.97 * 1;
        double sf2 = 0.95 * 1 * 0;
        double sf3 = 0.97 * 1;
        int n1 = (int)(2.0 * nCells);
        int n2 = (int)(0.5 * nCells);
        int n3 = (int)(2.0 * nCells);
        int n4 = (int)(0.2* nCells)*0 +1;
        int n5 = (int)(0.5* nCells);
        //var xNodes1 = GenericBlas.SinLinSpacing(xmin, -rAd, sf1, n1 + 1);
        //var xNodes2 = GenericBlas.SinLinSpacing(-rAd, rAd, sf2, n2 + 1);
        //var xNodes3 = GenericBlas.SinLinSpacing(rAd, xmax, sf3, n3 + 1);
        var wallNodes = (GenericBlas.Linspace(rAd, rAd + deltaNonDim, n4 + 1)).ToArray(); // nodes corresponding to the wall

        List<double> xNodes2 = (GenericBlas.SinLinSpacing(-rAd, rAd, sf2, n2 + 1)).ToList(); // nodes corresponding to the fuel inlet
        List<double> xNodes3 = (GenericBlas.SinLinSpacing(rAd, (xmax - rAd) * 2 + rAd, sf3, n1 * 2 + 1).ToList()); // Nodes corresponding to the oxidizer inlet, right part

        var myXnodes3 = xNodes3.GetSubVector(0, xNodes3.Count / 2 + 1); // Take only "left side" of node array

        for (int i = 0; i < myXnodes3.Count(); i++) { // Move values for accounting the wall width
            myXnodes3[i] = myXnodes3[i] + deltaNonDim;
        }

        var myxNodes1 = myXnodes3.CloneAs();
        myxNodes1.ScaleV(-1.0);
        Array.Reverse(myxNodes1);

        var wallNodesLeft = wallNodes.CloneAs();
        wallNodesLeft.ScaleV(-1.0);
        Array.Reverse(wallNodesLeft);

        List<double> list2 = new List<double>();
        list2.AddRange(myxNodes1.Take(n1 + 0).ToList());
        list2.AddRange(wallNodesLeft.Take(n4 + 0).ToList());
        list2.AddRange(xNodes2.Take(n2 + 0).ToList());
        list2.AddRange(wallNodes.Take(n4 + 0).ToList());
        list2.AddRange(myXnodes3.Take(n3 + 1).ToList());
        double[] _xNodes = list2.ToArray();


        int n6 =  (2 * nCells) * 3 + 1;
        var _yNodes = GenericBlas.SinLinSpacing(ymin, ymax * 2, stretchfactorY, (2 * nCells) * 3 + 1);
        var myYnodes = _yNodes.GetSubVector(0, _yNodes.Length / 2 + 1); // I just want a fine mesh in the bottom part of the grid.


        double tubeHeight = rAd * 4.0;

        // var wallNodesY = (GenericBlas.Linspace(0, tubeHeight, n5 + 1)).ToArray(); // nodes corresponding to the wall
        var wallNodesY = GenericBlas.SinLinSpacing(0, tubeHeight * 2, stretchfactorY, (2 * n5) * 1 + 1);
        var myYnodes1 = wallNodesY.GetSubVector(wallNodesY.Length / 2 ,wallNodesY.Length / 2+1   ); 
        for (int i = 0; i < myYnodes1.Count(); i++) { // Move values for accounting the wall height
            myYnodes1[i] = myYnodes1[i] - tubeHeight;
        }

        for (int i = 0; i < myYnodes.Count(); i++) { // Move values for accounting the wall height
            myYnodes[i] = myYnodes[i] + tubeHeight;
        }

        List<double> listY = new List<double>();
        listY.AddRange(myYnodes1.Take(n5 + 0).ToList());
        listY.AddRange(myYnodes.Take(n6/2 + 1).ToList());
       
        double[] _yNodes3 = listY.ToArray();
        double[] CutOut1Point1 = new double[2] { rAd, 0.0 };
        double[] CutOut1Point2 = new double[2] { rAd + deltaNonDim , tubeHeight};

        double[] CutOut2Point1 = new double[2] { -rAd, 0.0 };
        double[] CutOut2Point2 = new double[2] { -1*(rAd + deltaNonDim ), tubeHeight };

        var CutOut1 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut1.AddPoint(CutOut1Point1);
        CutOut1.AddPoint(CutOut1Point2);

        var CutOut2 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut2.AddPoint(CutOut2Point1);
        CutOut2.AddPoint(CutOut2Point2);
        var CutOuts = new BoSSS.Platform.Utils.Geom.BoundingBox[] { CutOut1, CutOut2 };

        var grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes3, CutOuts: CutOuts);
        grd.EdgeTagNames.Add(1, "Velocity_Inlet_O2");
        grd.EdgeTagNames.Add(2, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");
        // grd.EdgeTagNames.Add(4, "wall");
        grd.EdgeTagNames.Add(4, "NoSlipNeumann");
        // grd.EdgeTagNames.Add(4, "Velocity_Inlet_outer"); // We want a constant velocity field.

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //Edge tags
            //1: Velocity inlet O_2
            //2: Velocity inlet CH_4
            //3: Pressure outlet
            //4: NoSlipNeumann
            if(Math.Abs(x -  list2.Last() ) < 1e-8 || Math.Abs(x- list2.First() ) < 1e-8 )
            return 3;
            
            if(Math.Abs(y -(listY.ToList()).Last()) < 1e-8 )
            return 3;

            if (Math.Abs(y - ymin) < 1e-8) { // Bottom part
                if (Math.Abs(x - 0.0) < rAd + 1e-8)
                    return 2; // Velocity Inlet CH4
                if (x > wallNodes.Last() + 1e-8 || x < wallNodesLeft[0] + 1e-8) {
                    return 1;
                } 
            }

        
            return 4;

        }
        );
        
                // var gDat = new GridData(grd);
                // var em1 = gDat.GetBoundaryEdges();
                // em1.SaveToTextFile("alledges.csv", false, (double[] CoordGlobal, int LogicalItemIndex, int GeomItemIndex) => (double)gDat.iGeomEdges.EdgeTags[GeomItemIndex]);
        myDb.SaveGrid(ref grd);

        return grd;
        
    }
  
}

In [ ]:
// var grd = GridFactory.GenerateGrid_FullSystem(10);

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma) {
        using(var stw = new System.IO.StringWriter()) {

            stw.WriteLine("static class BoundaryValues {");

            stw.WriteLine("  static public double ConstantValue(double[] X) {");
            stw.WriteLine("    return "+ ConstVal +";");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuel(double[] X) {");
            stw.WriteLine("    return  (1.0 - Math.Pow(X[0] / "+inletRadius+", 2)) * "+uInFuel+" ;");
            stw.WriteLine("  }");

            // stw.WriteLine("  static public double ParabolaVelocityAir(double[] X) {");
            // stw.WriteLine("    return  -(1.0 - Math.Pow(X[1] / "+inletRadius+", 2)) * "+uInAir+";");
            // stw.WriteLine("  }");

            stw.WriteLine("  static public double RegularizedPlugFlowFuel(double[] X) {");    
            stw.WriteLine(" double res = 0;");  
            stw.WriteLine("         if(X[1] > 0) { ");
            stw.WriteLine("     double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] - "+inletRadius+"))); ");
            stw.WriteLine("           res = "+uInFuel+" * (1 - 2*H); ");
            stw.WriteLine("      }   ");
            stw.WriteLine("  else { ");
            stw.WriteLine("      double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] + ("+inletRadius+")))); ");
            stw.WriteLine("           res = "+uInFuel+" * ( 2*H-1); ");
            stw.WriteLine("  } ");
            stw.WriteLine("return res;"); 
            stw.WriteLine("  }");


            stw.WriteLine("  static public double RegularizedPlugFlowAir(double[] X) {");         
            stw.WriteLine(" double res = 0;");
            stw.WriteLine("         if(X[1] > 0) { ");
            stw.WriteLine("     double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] - "+inletRadius+"))); ");
            stw.WriteLine("           res = "+uInAir+" * (1 - 2*H)*(-1); ");
            stw.WriteLine("      }   ");
            stw.WriteLine("  else { ");
            stw.WriteLine("      double H = 0.5 * (1.0 + Math.Tanh("+sigma+" * (X[1] + ("+inletRadius+")))); ");
            stw.WriteLine("           res = "+uInAir+" * ( 2*H-1)*(-1); ");
            stw.WriteLine("  } ");
            stw.WriteLine("return res;"); 
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuelOscilatory(double[] X, double t) {");
            stw.WriteLine("    return  (1.0 + 0.1*Math.Sin(3.14*t) )*(1.0 - Math.Pow(X[0] / "+inletRadius+", 2)) * "+uInFuel+" ;");
            // stw.WriteLine("    return  (1.0 - Math.Pow(X[0] / "+inletRadius+", 2)) * "+uInFuel+" ;");

            stw.WriteLine("  }");

            stw.WriteLine("}"); 
            return stw.ToString();
        }
    }

    static public Formula Get_ConstantValue(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ConstantValue", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_ParabolaVelocityFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }
    // static public Formula Get_ParabolaVelocityAir(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
    //     return new Formula("BoundaryValues.ParabolaVelocityAir", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    // }

    static public Formula Get_RegularizedPlugFlowFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.RegularizedPlugFlowFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_RegularizedPlugFlowAir(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.RegularizedPlugFlowAir", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_ParabolaVelocityFuelOscilatory(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuelOscilatory",TimeDep:true, AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }
}

## Create base control file
In this ControlFile basic configuration of the CounterDiffusionFlame is defined.

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int nCells, bool isMF, double massFuelIn, double massAirIn, bool steady, double smoothfactor) {
    var CC = new ChemicalConstants();

    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();

    C.NumberOfChemicalSpecies = 4; 
    C.SetDGdegree(dg); //
    C.SetGrid(GridFactory.GenerateGrid_FullSystem(nCells));  //

    C.MatParamsMode = MaterialParamsMode.Sutherland;  //
    C.rhoOne = false; //!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    // Problem Definition
    //===================
    double TemperatureInFuel = 300;  //
    double TemperatureInOxidizer = 300; //
    double AtmPressure = 101325; // Pa
    double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };

    double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; //
    double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };

    double mwFuel = CC.getAvgMW(MWs, FuelInletConcentrations);
    double mwAir = CC.getAvgMW(MWs, OxidizerInletConcentrations);
    double densityAirIn = AtmPressure * mwAir / (CC.R_gas * TemperatureInOxidizer * 1000); // kg / m3
    double densityFuelIn = AtmPressure * mwFuel / (CC.R_gas * TemperatureInFuel * 1000); // kg / m3.
    double uInFuel = massFuelIn / densityFuelIn; //
    double uInAir = massAirIn / densityAirIn; //

    Console.WriteLine("VelocityFuel" + uInFuel);
    Console.WriteLine("VelocityAir" + uInAir);

    // Reference values
    //===================
    // Basic units to be used: Kg, m, s, mol, pa,
    double TRef = TemperatureInOxidizer;// Reference temperature  is the inlet temperature, (K)
    double pRef = AtmPressure; // Pa
    double uRef = Math.Max(uInFuel, uInAir); // m/s
    double r = 0.635 / 100; // Radius inner cylinder, m
    // double R = 2.54 / 100 *5; // external boundary... setting it really far away 

    double LRef = r;

    // C.SkipSolveAndEvaluateResidual = true; // !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 }; //No gravity.

    // Solver configuration
    // =======================
    C.smoothingFactor = smoothfactor;
    // C.NonLinearSolver.ConvergenceCriterion = 1e-8;
    // C.LinearSolver.ConvergenceCriterion = 1e-10;
    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.LinearSolver.SolverCode = LinearSolverCode.direct_pardiso;
    C.LinearSolver.verbose = false;
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.saveperiod = 1;
    C.PenaltyViscMomentum = 1.0; 
    C.PenaltyHeatConduction = 1.0;
    C.YFuelInlet = FuelInletConcentrations[0];
    C.YOxInlet = OxidizerInletConcentrations[1];
    C.FuelInletConcentrations = FuelInletConcentrations;
    C.OxidizerInletConcentrations = OxidizerInletConcentrations;
    C.TFuelInlet = 1.0;
    C.TOxInlet = 1.0;
    C.PhysicalParameters.IncludeConvection = true;
    // Chemical related parameters
    double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
    C.phi = s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
    var MLC = new MaterialLawCombustion(300, new double[] { }, C.MatParamsMode, C.rhoOne, true, 1.0, 1, 1, C.YOxInlet, C.YFuelInlet, C.zSt, CC, 0.75);
    var ThermoProperties = new ThermodynamicalProperties();

    //==========================
    //Derived reference values
    //==========================
    C.uRef = uRef; // Reference velocity
    C.LRef = LRef; // reference length
    C.pRef = AtmPressure; // reference pressure
    C.TRef = TemperatureInFuel;// reference temperature
    C.MWRef = MLC.getAvgMW(MWs, C.OxidizerInletConcentrations); // Air mean molecular weight
    C.rhoRef = C.pRef * C.MWRef / (8.314 * C.TRef * 1000); // Kg/m3. ok ;
    C.cpRef = 1.3;//ThermoProperties.Calculate_Cp_Mixture(new double[] { 0.23, 0.77 }, new string[] { "O2", "N2" }, 300); // 1.219185317353029;// Representative value, KJ/Kg K ========> 1.31 for the one-step kinetic model
    C.muRef = MLC.getViscosityDim(300);
    C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
    C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses
    C.T_ref_Sutherland = 300;
    double heatRelease_Ref = (C.TRef * C.cpRef);
    C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
    C.B = CC.PreExponentialFactor;

    C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };

    C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef);
    C.Reynolds = C.rhoRef * C.uRef * C.LRef / C.muRef;
    C.Prandtl = 0.75; 
    C.Schmidt = C.Prandtl; // Because Lewis number  is assumed as 1.0  (Le = Pr/Sc)
    // C.Lewis = new double[] { 0.97, 1.11, 1.39, 0.83, 1.0 };
    C.Lewis = new double[] { 1.0, 1.0, 1.0, 1.0, 1.0 };

    double g = 9.8; // m/s2
    C.Froude = Math.Sqrt(uRef * uRef / (C.LRef * g)); // Not used
    C.T_ref_Sutherland = 300; //////// Check this
    C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 }; 
    //==========================
    // Initial conditions
    //==========================
    double dummy = 0;
    double Radius = 0.5;
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef,dummy));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef,dummy));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_ConstantValue(0.0, Radius, uInFuel / C.uRef, uInAir / C.uRef,dummy));

    //==========================
    // Boundary conditions
    //==========================

    
    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    if(steady){
        C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ParabolaVelocityFuel( dummy, r/C.LRef, uInFuel / uRef, uInAir / uRef, dummy));
    } else{
        C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ParabolaVelocityFuelOscilatory( dummy, r/C.LRef, uInFuel / uRef, uInAir / uRef, dummy));
    }
    

    // (double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma) {
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue( uInAir / uRef, dummy, dummy,dummy, dummy));
    C.AddBoundaryValue("NoSlipNeumann");

    // C.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy,dummy, dummy));
    // C.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue( uInAir / uRef, dummy, dummy,dummy, dummy));

    return C;
}

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn, bool steady, double smoothfactor){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true,massFuelIn, massAirIn, steady, smoothfactor);
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "CoFlowFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells + "smoothfactor" + smoothfactor;
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.UseSelfMadeTemporalOperator = false;
    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 50; 
    
    // Boundary and initial conditions
    double dummy = -11111111;
    C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));        
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(0.0,dummy,dummy , dummy, dummy));
    
    C_MixtureFraction.NonLinearSolver.ConvergenceCriterion = 1e-4;
    C_MixtureFraction.AdaptiveMeshRefinement = true;
    C_MixtureFraction.AMR_startUpSweeps = 3;

    if(steady){
        int NoOfPseudoTimesteps =  2;
        C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 
        C_MixtureFraction.NoOfTimesteps = NoOfPseudoTimesteps ;
    } else{
        C_MixtureFraction.TimesteppingMode =  AppControl._TimesteppingMode.Transient; // Unsteady simulation...
        C_MixtureFraction.dtFixed = 0.01;
        C_MixtureFraction.NoOfTimesteps = 500;
    }
// Refinement

    double[] p1 = new double[2] { -3.0, 2.5 };
    double[] p2 = new double[2] { 3.0,6.0};
    var bb = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
    bb.AddPoint(p1);
    bb.AddPoint(p2);
    
    var ind =  new BoSSS.Application.XNSEC.AMRInBoundingBox(p1,p2);
    ind.maxRefinementLevel = 2;
    C_MixtureFraction.activeAMRlevelIndicators.Add(ind);

// C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 1;
// C_MixtureFraction.NonLinearSolver.MinSolverIterations= 1;
    // C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onProblematicPoints(troubledPoints,C_MixtureFraction.AMR_startUpSweeps) );  
    // C_MixtureFraction.activeAMRlevelIndicators.Add(new BoSSS.Application.XNSEC.AMR_RefineAroundProblematicPoints(troubledPoints, 3, 0.2));
    C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_MixtureFraction.zSt,3) );
    return C_MixtureFraction;
}

## Send and run jobs

In [ ]:
foreach(double smoothfactor in smoothingFactors){
    Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
    var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dgMF, nCells, InitialMassFuelIn, InitialMassAirIn,true,smoothfactor); 
    string jobName       = C_MixtureFraction.SessionName;
    Console.WriteLine(jobName);
    var oneJob           = new Job(jobName, solver_MF);
    oneJob.NumberOfMPIProcs = NumberOfProcMF;
    oneJob.NumberOfThreads = 1;
    oneJob.SetControlObject(C_MixtureFraction);
    oneJob.Activate(myBatch); 
    }


In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

## Starting the finite-rate chemistry simulation

Now that the simulation for an "infinite" reaction rate is done, we use it for initializing the system with finite reaction rate. The goal is to obtain solutions of the counter difussion flame for increasing strain values. We start with a low strain (bigger Dahmkoehler number), which is increased until extintion is (hopefully) found

In [ ]:
static XNSEC_Control GiveMeTheFullCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn, ISessionInfo SessionToRestart, double smoothfactor, bool steady) {
    var C_OneStep = GiveMeTheCtrlFile(dg, nCells, false, massFuelIn, massAirIn, steady, smoothfactor);
    C_OneStep.physicsMode = PhysicsMode.Combustion;
    C_OneStep.ProjectName = "CoFlowFlame";
    int std = steady == true ? 0 : 1;
    string name = C_OneStep.ProjectName + "P" + dg + "K" + nCells + "smoothfactor" + smoothfactor;
    C_OneStep.SessionName = "Full_" + name;
    C_OneStep.VariableOneStepParameters = true;
    C_OneStep.UseSelfMadeTemporalOperator = false;
    C_OneStep.myThermalWallType = SIPDiffusionTemperature.ThermalWallType.Adiabatic;
    C_OneStep.Timestepper_LevelSetHandling = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    C_OneStep.UseMixtureFractionsForCombustionInitialization = true;

    C_OneStep.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
    C_OneStep.LinearSolver.NoOfMultigridLevels = 4;
    C_OneStep.ChemicalReactionActive = true;
    C_OneStep.AdaptiveMeshRefinement = true;

    // C_OneStep.HeatCapacityMode = MaterialLaw_MultipleSpecies.CpCalculationMode.mixture;

 
    C_OneStep.NonLinearSolver.MaxSolverIterations = 20;
    if (steady) {
        C_OneStep.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    } else {
        C_OneStep.TimesteppingMode = AppControl._TimesteppingMode.Transient; // Unsteady simulation...
        C_OneStep.dtFixed = 0.01;
        C_OneStep.NoOfTimesteps = 1000;
    }
    C_OneStep.NoOfTimesteps = 5;
    C_OneStep.AMR_startUpSweeps = 2;
    if (C_OneStep.ChemicalReactionActive) {
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_onReactiveZones(C_OneStep.MolarMasses, 2, 0.1));
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnVariableLimits("Temperature", new double[] { -100, 4 },3)); // Refine all cells with T > 5 (and T < -100)
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnFieldGradient(2, 0.1, VariableNames.Temperature));
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 2));
    }
    // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 3));
    // C_OneStep.NonLinearSolver.MaxSolverIterations = 10;

    // limiting of variable values
    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    double eps = 1e-2;
    Bounds.Add(VariableNames.Temperature, new Tuple<double, double>(1.0 - eps, 10)); // Min temp should be the inlet temperature.
    Bounds.Add(VariableNames.MassFraction0, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4)); // Between 0 and 1 per definition
    Bounds.Add(VariableNames.MassFraction1, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction2, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction3, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    C_OneStep.VariableBounds = Bounds;
    // Boundary conditions

    double dummy = 0;
    if (SessionToRestart != null) {
        C_OneStep.SetRestart(SessionToRestart);
    } else {
        C_OneStep.AddInitialValue(VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(0.23, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    }

    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[0], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[1], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[2], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[3], dummy, dummy, dummy, dummy));

    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[0], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[1], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[2], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[3], dummy, dummy, dummy, dummy));

    return C_OneStep;
}

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);

Calculate the full solution for the initial value

In [ ]:
if(!calculateOnlyMixtureFraction){
    foreach(double smoothfactor  in smoothingFactors){
        var sess = (myDb.Sessions.Where(s => s.Name == "FS_CoFlowFlameP" + dgMF + "K" + nCells+ "smoothfactor" + smoothfactor)).FirstOrDefault();
        if(sess != null){
        Console.WriteLine("Restarting from session with name " + sess.Name); 
        } else{
        Console.WriteLine("No session found for restart. Starting simulation without combustion");
        }
        var C = GiveMeTheFullCtrlFile(dgFull, nCells, InitialMassFuelIn, InitialMassAirIn, sess, smoothfactor, true);
        string jobName = C.SessionName;
        Console.WriteLine(jobName);
        var oneJob = new Job(jobName, solver);
        oneJob.NumberOfMPIProcs = 8;
        oneJob.NumberOfThreads = 1;
        oneJob.SetControlObject(C);
        oneJob.Activate(myBatch);
    }
}

In [ ]:
// wait for all jobs to finish (up to 2 days, check every 10 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

In [ ]:
// myDb.Sessions[0].Export().Do()

In [ ]:
// int counter = 0;
// if(!calculateOnlyMixtureFraction){
// foreach (double xDensityNodes in xCellsMultiplier){
//     foreach (int nCells in nCellsArray) {
//         foreach (int dg in DGDegrees) {
//             foreach (int i in multiplierS) {
//                 var sess = myDb.Sessions.Where(s=>s.Name.Contains("Full")).SingleOrDefault();
//                 if(sess != null){
//                     Console.WriteLine("Restarting from session with name " + sess.Name); 
//                 } else{
//                     Console.WriteLine("No session found for restart. ");
//                 }
//                 var C = GiveMeTheFullCtrlFile(dg, nCells, InitialMassFuelIn * i, InitialMassAirIn * i, sess, parabolicVelocityProfile, chemicalReactionActive, useFullGeometry, wallBounded, i, counter, xDensityNodes, false);
//                 string jobName = C.SessionName;
//                 Console.WriteLine(jobName);
//                 var oneJob = new Job(jobName, solver);
//                 oneJob.NumberOfMPIProcs = 12;
//                 oneJob.SetControlObject(C);
//                 oneJob.Activate(myBatch);
//                 counter++;
//             }
//         }
//     }
// }
// }